<a href="https://colab.research.google.com/github/SalmanOkz/Ai-Based-Cheating-Plagiarism-Detection/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.config/kaggle

# Move kaggle.json to the .kaggle directory
!mv /content/kaggle.json ~/.config/kaggle/

# Set permissions for kaggle.json
!chmod 600 ~/.config/kaggle/kaggle.json

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhruv413/mpiigaze")

print("Path to dataset files:", path)

100%|██████████| 2.08G/2.08G [00:28<00:00, 78.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dhruv413/mpiigaze/versions/1


In [5]:
path

'/root/.cache/kagglehub/datasets/dhruv413/mpiigaze/versions/1'

In [6]:
import os

# List the contents of the dataset directory
print(os.listdir(path))

['MPIIGaze']


In [15]:
!pip install mediapipe opencv-python-headless

In [17]:
!pip uninstall -y mediapipe
!pip install mediapipe

Found existing installation: mediapipe 0.10.31
Uninstalling mediapipe-0.10.31:
  Successfully uninstalled mediapipe-0.10.31
  Using cached mediapipe-0.10.31-py3-none-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
Using cached mediapipe-0.10.31-py3-none-manylinux_2_28_x86_64.whl (10.3 MB)


In [3]:
import mediapipe as mp
print(np.__path__)

['/usr/local/lib/python3.12/dist-packages/numpy']


In [4]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

def get_gaze_ratio(eye_points, facial_landmarks, img):
    # Ankh ke landmarks nikalna
    left_side = np.array([facial_landmarks.landmark[eye_points[0]].x * img.shape[1],
                          facial_landmarks.landmark[eye_points[0]].y * img.shape[0]])
    right_side = np.array([facial_landmarks.landmark[eye_points[1]].x * img.shape[1],
                           facial_landmarks.landmark[eye_points[1]].y * img.shape[0]])

    # Iris (Center) landmark nikalna
    center_iris = np.array([facial_landmarks.landmark[eye_points[2]].x * img.shape[1],
                            facial_landmarks.landmark[eye_points[2]].y * img.shape[0]])

    # Distance calculate karna
    total_width = np.linalg.norm(left_side - right_side)
    iris_distance = np.linalg.norm(left_side - center_iris)

    return iris_distance / total_width

# Test on one image from your dataset
image_path = "/root/.cache/kagglehub/datasets/dhruv413/mpiigaze/versions/1" # Apna path dein
image = cv2.imread(image_path)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = face_mesh.process(rgb_image)

if results.multi_face_landmarks:
    landmarks = results.multi_face_landmarks[0]

    # Right Eye: Inner Corner (133), Outer (33), Iris Center (468)
    ratio = get_gaze_ratio([133, 33, 468], landmarks, image)

    if ratio < 0.35: gaze = "Looking Right"
    elif ratio > 0.65: gaze = "Looking Left"
    else: gaze = "Center"

    print(f"Detected Gaze: {gaze} (Ratio: {ratio:.2f})")
    cv2_imshow(image)

AttributeError: module 'mediapipe' has no attribute 'solutions'